In [2]:
!pip3 install html5lib
import requests
from bs4 import BeautifulSoup
import pandas as pd
!pip3 install sodapy
from sodapy import Socrata

In [3]:
#data scraping
client = Socrata("data.montgomerycountymd.gov", None)

In [6]:
results = client.get("4mse-ku6q",limit=100000)
df = pd.DataFrame.from_records(results)
df

,seq_id,date_of_stop,time_of_stop,agency,subagency,description,location,latitude,longitude,accident,...,:@computed_region_kbsp_ykn9,:@computed_region_d7bw_bq6x,:@computed_region_rbt8_3x7n,search_conducted,search_outcome,search_reason_for_stop,search_disposition,search_reason,search_type,search_arrest_reason
0,24e54d0a-8538-4425-a7b1-7d682c7647bb,2020-11-01T00:00:00.000,22:00:00,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVER SPINNING WHEELS,CENTERWAY RD / GOSHEN,39.1697183333333,-77.1935133333333,No,...,38,28,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24e54d0a-8538-4425-a7b1-7d682c7647bb,2020-11-01T00:00:00.000,22:00:00,MCP,"6th District, Gaithersburg / Montgomery Village",DRIVING VEHICLE IN EXCESS OF REASONABLE AND PR...,CENTERWAY RD / GOSHEN,39.1697183333333,-77.1935133333333,No,...,38,28,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,88c23ddb-96f9-4ef4-9e9b-876a5c57b3ce,2020-11-01T00:00:00.000,21:44:00,MCP,"1st District, Rockville",DRIVER FAILURE TO STOP AT STOP SIGN LINE,W MONTGOMERY/ SHADY GROVE,39.1021966666667,-77.1885333333333,No,...,25,71,12,No,Warning,21-707(a),NaN,NaN,NaN,NaN
3,88c23ddb-96f9-4ef4-9e9b-876a5c57b3ce,2020-11-01T00:00:00.000,21:44:00,MCP,"1st District, Rockville",DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...,W MONTGOMERY/ SHADY GROVE,39.1021966666667,-77.1885333333333,No,...,25,71,12,No,Warning,21-707(a),NaN,NaN,NaN,NaN
4,8620d5b5-98e5-4175-ab6f-57def29f5e2b,2020-11-01T00:00:00.000,21:43:00,MCP,"3rd District, Silver Spring",DRIVING VEH. W/O ADEQUATE REAR REG. PLATE ILLU...,NEW HAMPSHIRE AVE AND CHALMERS RD,39.0278766666667,-76.9825483333333,No,...,18,15,1,Yes,Warning,22-204(f),Nothing,Probable Cause,Both,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,c26d0f88-ab7d-471e-801a-6801b537bd39,2019-09-26T00:00:00.000,07:30:00,MCP,"2nd District, Bethesda",DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...,EDSON LN @ WOODGLEN DR,39.0415283333333,-77.1132,No,...,19,60,1,No,Warning,21-201(a1),NaN,NaN,NaN,NaN
99996,590efc45-5508-41cc-9ad1-3340ce0d70c1,2019-09-26T00:00:00.000,07:25:00,MCP,"2nd District, Bethesda",DRIVER FAILURE TO STOP AT STEADY CIRCULAR RED ...,W CEDDAR LN AT OLD GEORGETOWN RD,39.0038033333333,-77.11052,No,...,12,103,1,No,Warning,21-202(h1),NaN,NaN,NaN,NaN
99997,590efc45-5508-41cc-9ad1-3340ce0d70c1,2019-09-26T00:00:00.000,07:25:00,MCP,"2nd District, Bethesda",DRIVER CHANGING LANES WHEN UNSAFE,W CEDDAR LN AT OLD GEORGETOWN RD,39.0038033333333,-77.11052,No,...,12,103,1,No,Warning,21-202(h1),NaN,NaN,NaN,NaN
99998,0a482472-ad58-4fcf-bc98-9514adf07ed5,2019-09-26T00:00:00.000,07:11:00,MCP,"1st District, Rockville",STOP LIGHTS (*),JONES LANE AT FALCONBRIDGE DRIVE,39.0980433333333,-77.2654816666667,No,...,28,33,1,No,Warning,21-707(a),NaN,NaN,NaN,NaN
